In [17]:
include("./src/parse_file.jl")

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [18]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"


"/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"

In [19]:
results = analyze_file(path)

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [26]:
x= results["chr1"]

27×631931 SparseMatrixCSC{Int64, Int64} with 105 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠈⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠒⠂⠐

In [37]:
test = x[:,136617:136620]

27×4 SparseMatrixCSC{Int64, Int64} with 7 stored entries:
⢨⢨
⠀⠈
⠀⠀
⠀⠀
⠀⠀
⠀⠀
⠀⠀

In [58]:
function get_unsparse_nonzero_rows(matrix::SparseMatrixCSC{Int64, Int64})
	nzrows = unique(matrix.rowval)
	if !isempty(nzrows)
		return Matrix(matrix)[nzrows,:]
	else 
		return Matrix{Int64}(0, (0, 0))
	end
end

get_unsparse_nonzero_rows (generic function with 1 method)

In [87]:
get_unsparse_nonzero_rows(x[:,1:1000])

MethodError: MethodError: no method matching Matrix{Int64}(::Int64, ::Tuple{Int64, Int64})
Closest candidates are:
  Matrix{T}(!Matched::LinearAlgebra.UniformScaling, ::Tuple{Int64, Int64}) where T at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:496
  Array{T, N}(!Matched::Nothing, ::Any...) where {T, N} at baseext.jl:42
  Array{T, N}(!Matched::Missing, ::Any...) where {T, N} at baseext.jl:43
  ...

In [39]:
b = get_unsparse_nonzero_rows(test)

4×4 Matrix{Int64}:
 0  77  0  77
 0  77  0  77
 0  77  0  77
 0   0  0  85

In [62]:
function matrix_to_epiallele_vector(mat::Matrix{Int64})
	vec = Vector{Epiallele}(undef, size(mat, 1))
	for i in 1:size(mat, 1)
		vec[i] = Epiallele(mat[i,:])
	end
	return vec
end


matrix_to_epiallele_vector (generic function with 1 method)

In [63]:
matrix_to_epiallele_vector(b)

4-element Vector{Epiallele}:
 [0, 77, 0, 77]
 [0, 77, 0, 77]
 [0, 77, 0, 77]
 [0, 0, 0, 85]

In [149]:
a = [0 77 0 78; 0 77 15 85; 0 78 0 85]
a_vec = matrix_to_epiallele_vector(a)
display(a)

3×4 Matrix{Int64}:
 0  77   0  78
 0  77  15  85
 0  78   0  85

In [151]:
count_epialleles(a_vec)

3-element Vector{Epiallele}:
 [0, 0, 0, 0]
 [0, 0, 0, 0]
 [0, 0, 0, 0]

3-element Vector{Epiallele}:
 [0, 77, 0, 78]
 [0, 0, 0, 0]
 [0, 0, 0, 0]

3-element Vector{Epiallele}:
 [0, 77, 0, 78]
 [0, 77, 15, 85]
 [0, 0, 0, 0]

[0, 77, 0, 78] is different from all previously seen alleles
[0, 77, 15, 85] is different from all previously seen alleles
seen [0, 78, 0, 85] already


2

In [140]:
function diff_from_all_epialleles(needle::Epiallele, haystack::Vector{Epiallele})
	for i in 1:length(haystack)
		if needle == haystack[i]
			return false
		end
	end
	return true
end

diff_from_all_epialleles (generic function with 1 method)

In [153]:
function count_epialleles(epiallele_vec::Vector{Epiallele})
	seen = fill(zeros(Epiallele, length(epiallele_vec[1])), length(epiallele_vec))
	unique = 0
	for i in 1:length(epiallele_vec)
		diff_from_all_epialleles(epiallele_vec[i], seen) && (unique += 1)
		seen[i] = epiallele_vec[i]
	end
	return unique
end

count_epialleles (generic function with 1 method)

In [156]:
window_size = 4

for i in 1:window_size:200000
	try
		a = get_unsparse_nonzero_rows(x[:,i:i+window_size-1])
		if !isempty(a)
			@show a
			n = count_epialleles(matrix_to_epiallele_vector(a))
			println("found $n alleles")
		end
	catch
	end
end

a = [0 0 77 0]
found 1 alleles
a = [0 0 0 77]
found 1 alleles
a = [0 77 0 77]
found 1 alleles
a = [0 0 0 77]
found 1 alleles
a = [77 0 0 0]
found 1 alleles
a = [0 77 0 0]
found 1 alleles
a = [0 0 0 77]
found 1 alleles
a = [0 77 0 0]
found 1 alleles
a = [0 0 77 0]
found 1 alleles
a = [0 0 77 0; 0 0 77 0; 0 0 77 0]
found 1 alleles
a = [0 77 0 77; 0 77 0 77; 0 77 0 77; 0 0 0 85]
found 2 alleles
a = [0 0 0 77; 0 0 0 77; 0 0 0 77; 0 0 0 77]
found 1 alleles
a = [0 0 0 85]
found 1 alleles
a = [0 85 0 0]
found 1 alleles
a = [0 77 0 0; 0 77 0 0]
found 1 alleles
a = [85 0 0 0; 85 0 0 0]
found 1 alleles


a = [0 0 0 77]
found 1 alleles
a = [0 85 0 0]
found 1 alleles
a = [0 77 0 0]
found 1 alleles
a = [0 77 0 0; 0 77 0 0]
found 1 alleles


a = [0 85 0 0]
found 1 alleles
a = [0 77 0 0; 0 77 0 0]
found 1 alleles
a = [0 0 77 0; 0 0 77 0]
found 1 alleles


In [157]:
Char(85)

'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)

In [78]:
#= We need to set up this struct to allow for quick comparisons between epireads. 
	If they only differ at 'N' bases, they should be treated as identical =#

struct Epiallele <: AbstractVector{Int64}
	x::Vector{Int64}
end

function Base.size(x::Epiallele)
	return size(x.x)
end

function Base.getindex(x::Epiallele, i::Int64)
	return x.x[i]
end

function Base.:(==)(x::Epiallele, y::Epiallele) 
	# if the epialleles are different lengths, return false
	if length(x.x) != length(y.x)
		return false
	end
	# compare elementwise; if only difference is at N's (78), return true
	for i in 1:length(x.x)
		if x.x[i] != y.x[i] && x.x[i] != 78 && y.x[i] != 78
			return false
		end
	end
	return true
end

function Base.isequal(x::Epiallele, y::Epiallele)
	return x == y
end

# Zero initializer for Epiallele
function Base.zeros(::Type{Epiallele}, n::Int64)
	return Epiallele(zeros(Int64, n))
end


In [10]:
test = Epiallele([0, 78, 0, 75, 0])
test2 = Epiallele([0, 0, 0, 78])
test3 = Epiallele([0, 0, 0, 0])


4-element Epiallele:
 0
 0
 0
 0

2-element Vector{Epiallele}:
 [0, 0, 0, 0]
 [0, 0, 0, 0]

In [ ]:
#= What do we do with different characters that could be in the results?
	M/U - keep
	O/S - ignore for now
	ACGT - inform different genetic alleles, keep 
	N - specific to the read; a better read of the same locus may have a base here. Treat as equivalent to 0 and ACGT
	Q - interesting but doesn't pertain to epialleles. Discard. 

# of epialleles is sum of different column values, minus number of N values
=#

In [7]:
Int.(only.(collect("MUOSACGTNQ")))

10-element Vector{Int64}:
 77
 85
 79
 83
 65
 67
 71
 84
 78
 81

In [13]:
using BenchmarkTools

In [94]:
Array

Array

In [73]:
zeros(Epiallele, 4)

4-element Epiallele:
 0
 0
 0
 0